## Runnables

### RunnableMap

In [1]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

def mul_three(x: int) -> int:
    return x * 3

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
runnable_3 = RunnableLambda(mul_three)

sequence = runnable_1 | {  # this dict is coerced to a RunnableParallel
    "mul_two": runnable_2,
    "mul_three": runnable_3,
}
# Or equivalently:
# sequence = runnable_1 | RunnableParallel(
#     {"mul_two": runnable_2, "mul_three": runnable_3}
# )
# Also equivalently:
# sequence = runnable_1 | RunnableParallel(
#     mul_two=runnable_2,
#     mul_three=runnable_3,
# )

sequence.invoke(1)
await sequence.ainvoke(1)

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

[{'mul_two': 4, 'mul_three': 6},
 {'mul_two': 6, 'mul_three': 9},
 {'mul_two': 8, 'mul_three': 12}]

In [4]:
from langchain_core.outputs import Generation
from langchain_core.output_parsers import JsonOutputParser

# Create a JsonOutputParser instance
parser = JsonOutputParser()

# Assume we have a Generation object
generation = Generation(text='{"key": "value" ')

# Parse the result
result = parser.parse_result([generation])

# result now contains the parsed JSON object
print(result)  # Output: {'key': 'value'}

{'key': 'value'}


In [7]:
from typing import List
from langchain_core.exceptions import OutputParserException
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")

parser = PydanticOutputParser(pydantic_object=Actor)
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}" # Example of malformed output

try:
    parser.parse(misformatted)
except OutputParserException as e:
    print(e)

Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [9]:
import os
import time
import torch
from pathlib import Path
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from langchain_huggingface.llms import HuggingFacePipeline


def load_llm(model_name):
    token_path = Path("token.txt")
    if not token_path.exists():
        raise FileNotFoundError("Missing HuggingFace token.txt")

    with token_path.open("r") as f:
        hf_token = f.read().strip()

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        low_cpu_mem_usage=True,
        device_map="auto",
        token=hf_token
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    model_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id,
        device_map="auto"
    )

    return HuggingFacePipeline(pipeline=model_pipeline)

MODEL_NAME= "google/gemma-2b-it"
# MODEL_NAME = "lmsys/vicuna-7b-v1.5"
llm = load_llm(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [13]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"  # Example of malformed output

try:
    result = new_parser.parse(misformatted)
    print(result)
except OutputParserException as e:
    print(f"Failed to fix the output: {e}")
    print("Troubleshooting details:")
    print("Ensure the malformed output matches the expected schema for the Actor model.")
    print("Expected fields: 'name' (string) and 'film_names' (list of strings).")

Failed to fix the output: Failed to parse Actor from completion {"properties": {"name": {"description": "name of an actor", "title": "Name", "type": "string"}, "film_names": {"description": "list of names of films they starred in", "items": {"type": "string"}, "title": "Film Names", "type": "array"}}, "required": ["name", "film_names"]}. Got: 2 validation errors for Actor
name
  Field required [type=missing, input_value={'properties': {'name': {... ['name', 'film_names']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
film_names
  Field required [type=missing, input_value={'properties': {'name': {... ['name', 'film_names']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Troubleshooting details:
Ensure the malformed output matches the expected schema for the Actor model.
Expected field